In [1]:
import pandas as pd

In [2]:
detail_dfs = {
    "2009": "Zbp09detail.txt",
    "2010": "zbp10detail.txt",
    "2011": "zbp11detail.txt",
    "2012": "zbp12detail.txt",
    "2013": "zbp13detail.txt",
    "2014": "zbp14detail.txt",
    "2015": "zbp15detail.txt",
    "2016": "zbp16detail.txt"
}

total_dfs = {
    "2009": "zbp09totals.txt",
    "2010": "zbp10totals.txt",
    "2011": "zbp11totals.txt",
    "2012": "zbp12totals.txt",
    "2013": "zbp13totals.txt",
    "2014": "zbp14totals.txt",
    '2015': "zbp15totals.txt",
    "2016": "zbp16totals.txt"
}

In [3]:
for i in detail_dfs.keys():
    filename = detail_dfs[i]
    detail_dfs[i] = pd.read_table("~/real_estate/data/raw/cbp/"+filename, sep=",")
    detail_dfs[i]["year"] = int(i)

In [4]:
for i in total_dfs.keys():
    filename = total_dfs[i]
    total_dfs[i] = pd.read_table("~/real_estate/data/raw/cbp/"+filename, sep=",")
    total_dfs[i]["year"] = int(i)

In [8]:
dfs = []

In [9]:
for i in detail_dfs.keys():
    
    df_detail = detail_dfs[i]
    df_total = total_dfs[i]
    
    # Filter
    df_general = df_detail.loc[df_detail["naics"] == "------"]
    df_restaurants = df_detail.loc[df_detail["naics"] == "7225//"]
    df_bars = df_detail.loc[df_detail["naics"] == "7224//"]
    
    # Rename stuff
    df_restaurants = df_restaurants.rename(columns={"est":"restaurant_count"})
    df_bars = df_bars.rename(columns={"est":"bar_count"})
    
    # Create vars
    biz_100_249 = df_general["n100_249"].values
    biz_250_499 = df_general["n250_499"].values
    biz_500_999 = df_general["n500_999"].values
    biz_1000 = df_general["n1000"].values
    
    # Add columns
    df_general.loc[:,"business_100+"] = biz_100_249 + biz_250_499 + biz_500_999 + biz_1000
    df_general.loc[:,"business_500+"] = biz_500_999 + biz_1000
    
    # Merge
    df_ = pd.merge(df_total[["zip","est","emp","year"]], df_general[["zip","business_100+","business_500+"]], on="zip", how="left")
    df_ = pd.merge(df_, df_restaurants[["zip","restaurant_count"]], on="zip", how="left")
    df_ = pd.merge(df_, df_bars[["zip","bar_count"]], on="zip", how="left")
    
    dfs.append(df_)

/home/gnazareths/.local/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/gnazareths/.local/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
df = pd.concat(dfs, axis=0)

In [14]:
df["year"] = df["year"].values + 1
df = df.rename(columns={"est":"business_count_cbp_lag",
                        "emp":"employment_count_cbp_lag",
                        "business_100+":"business_100_count_cbp_lag",
                        "business_500+":"business_500_count_cbp_lag",
                        "restaurant_count":"restaurant_count_cbp_lag",
                        "bar_count":"bar_count_cbp_lag",
                        "zip":"postal_code"})

In [16]:
df = df.fillna(value=0)

In [20]:
df.head()

,postal_code,business_count_cbp_lag,employment_count_cbp_lag,year,business_100_count_cbp_lag,business_500_count_cbp_lag,restaurant_count_cbp_lag,bar_count_cbp_lag
0,501,2,0,2014,0,0,0.0,0.0
1,1001,439,9418,2014,17,1,36.0,5.0
2,1002,572,17263,2014,8,2,57.0,4.0
3,1003,15,184,2014,0,0,0.0,0.0
4,1004,16,179,2014,1,0,3.0,0.0


In [21]:
df.to_csv("~/real_estate/data/v9/cbp_zipcode_df.csv", index=False)